In [3]:
#!/usr/bin/env python
# coding: utf-8
# author: Bo Tang

from collections import namedtuple
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from sklearn import tree

class optimalDecisionTreeClassifier:
    """
    Optimal regression tree with an integrated linear term (IORFA).
    Prediction: f(x) = x @ beta + gamma_leaf.
    """
    def __init__(self, max_depth=3, min_samples_split=2, alpha=0, warmstart=True, timelimit=600, output=True,
                 gamma_bounds=(-10000, 10000)):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.alpha = alpha
        self.warmstart = warmstart
        self.timelimit = timelimit
        self.output = output
        self.gamma_bounds = gamma_bounds
        self.trained = False
        self.optgap = None
        self.feature_names_ = None
        self.feature_mins_ = None
        self.feature_maxs_ = None
        self.big_m_ = None
        self._beta = None
        self._gamma = None

        # node index
        self.n_index = [i+1 for i in range(2 ** (self.max_depth + 1) - 1)]
        self.b_index = self.n_index[:-2**self.max_depth] # branch nodes
        self.l_index = self.n_index[-2**self.max_depth:] # leaf nodes

    def fit(self, x, y):
        """
        fit training data
        """
        if hasattr(x, "columns"):
            self.feature_names_ = list(x.columns)
        x = np.asarray(x, dtype=float)
        y = np.asarray(y, dtype=float).ravel()

        # data size
        self.n, self.p = x.shape
        self.feature_mins_ = np.min(x, axis=0)
        self.feature_maxs_ = np.max(x, axis=0)
        if self.output:
            print('Training data include {} instances, {} features.'.format(self.n,self.p))

        # solve MIP
        self.m, self.a, self.b, self.d, self.l, self.beta, self.gamma = self._buildMIP(x, y)
        
        if self.warmstart:
            self._setStart(x, y, self.a, self.b, self.d, self.l)

        self.m.optimize()
        self.optgap = self.m.MIPGap

        # get parameters
        self._a = {ind:self.a[ind].x for ind in self.a}
        self._b = {ind:self.b[ind].x for ind in self.b}
        self._d = {ind:self.d[ind].x for ind in self.d}
        self._beta = np.array([self.beta[p].x for p in range(self.p)])
        self._gamma = {t:self.gamma[t].x for t in self.l_index}

        self.trained = True

    def predict(self, x):
        """
        model prediction
        """
        if not self.trained:
            raise AssertionError('This optimalDecisionTreeClassifier instance is not fitted yet.')
        x = np.asarray(x, dtype=float)
        if x.ndim == 1:
            x = x.reshape(1, -1)
        if x.shape[1] != self.p:
            raise ValueError(f'Expected {self.p} features, got {x.shape[1]}.')

        y_pred = np.zeros(x.shape[0], dtype=float)
        for i, xi in enumerate(x):
            t = 1
            while t not in self.l_index:
                if self._d.get(t, 0.0) < 0.5:
                    t = 2 * t + 1
                    continue
                split_val = sum(self._a[j, t] * xi[j] for j in range(self.p))
                if split_val + 1e-9 >= self._b[t]:
                    t = 2 * t + 1
                else:
                    t = 2 * t
            y_pred[i] = float(np.dot(xi, self._beta) + self._gamma[t])

        return y_pred

    def _buildMIP(self, x, y):
        """
        build MIP formulation for Optimal Decision Tree
        """
        # create a model
        m = gp.Model('m')

        # output
        m.Params.outputFlag = self.output
        m.Params.LogToConsole = self.output
        # time limit
        m.Params.timelimit = self.timelimit
        # parallel
        m.params.threads = 0

        # model sense
        m.modelSense = GRB.MINIMIZE

        # variables
        a = m.addVars(self.p, self.b_index, vtype=GRB.BINARY, name='a') # splitting feature
        b = m.addVars(self.b_index, vtype=GRB.CONTINUOUS, name='b') # splitting threshold
        d = m.addVars(self.b_index, vtype=GRB.BINARY, name='d') # splitting option
        z = m.addVars(self.n, self.l_index, vtype=GRB.BINARY, name='z') # leaf node assignment
        l = m.addVars(self.l_index, vtype=GRB.BINARY, name='l') # leaf node activation
        beta = m.addVars(self.p, vtype=GRB.CONTINUOUS, name='beta')
        varkappa = m.addVars(self.n, self.l_index, vtype=GRB.CONTINUOUS, name='varkappa')
        gamma = m.addVars(self.l_index, vtype=GRB.CONTINUOUS, name='gamma')
        lamb = m.addVars(self.n, vtype=GRB.CONTINUOUS, name='f')
        N = m.addVars(self.l_index, vtype=GRB.CONTINUOUS, name='N') # leaf node samples

        # calculate minimum distance
        min_dis = self._calMinDist(x)

        feature_min = self.feature_mins_ if self.feature_mins_ is not None else np.min(x, axis=0)
        feature_max = self.feature_maxs_ if self.feature_maxs_ is not None else np.max(x, axis=0)
        feature_range = feature_max - feature_min
        max_range = float(np.max(feature_range)) if feature_range.size else 0.0
        eps_max = float(np.max(min_dis)) if len(min_dis) > 0 else 0.0
        big_m = max_range + eps_max
        if big_m <= 0:
            big_m = 1.0
        self.big_m_ = big_m

        objExp = gp.QuadExpr()

        self.Lower, self.Upper = self.gamma_bounds

        # add single terms using add
        for i in range(self.n):
            var = y[i] - gp.quicksum(x[i, p] * beta[p] for p in range(self.p)) - lamb[i]

            objExp.add(var * var) 
            
            m.addConstr(lamb[i] == gp.quicksum(varkappa[i, t] for t in self.l_index))
            
            for t in self.l_index:
                m.addConstr(self.Lower*z[i, t] <= varkappa[i, t])
                m.addConstr(varkappa[i, t] <= self.Upper*z[i, t])
                m.addConstr(self.Lower*(1-z[i, t]) <= gamma[t]-varkappa[i, t])
                m.addConstr(gamma[t]-varkappa[i, t] <= self.Upper*(1-z[i, t]))
                
            # gp.quicksum(gamma[t]*z[i, t] for t in self.l_index))

        complexity = gp.quicksum(d[t] for t in self.b_index)
        m.setObjective((1.0 / self.n) * objExp + self.alpha * complexity)

        # (16)
        m.addConstrs(z.sum('*', t) == N[t] for t in self.l_index)
        # (13) and (14)
        for t in self.l_index:
            left = (t % 2 == 0)
            ta = t // 2
            while ta != 0:
                if left:
                    m.addConstrs(gp.quicksum(a[j,ta] * (x[i,j] + min_dis[j]) for j in range(self.p))
                                 +
                                 big_m * (1 - d[ta])
                                 <=
                                 b[ta] + big_m * (1 - z[i,t])
                                 for i in range(self.n))
                else:
                    m.addConstrs(gp.quicksum(a[j,ta] * x[i,j] for j in range(self.p))
                                 >=
                                 b[ta] - big_m * (1 - z[i,t])
                                 for i in range(self.n))
                left = (ta % 2 == 0)
                ta //= 2

        # (8)
        m.addConstrs(z.sum(i, '*') == 1 for i in range(self.n))
        # (6)
        m.addConstrs(z[i,t] <= l[t] for t in self.l_index for i in range(self.n))
        # (7)
        m.addConstrs(z.sum('*', t) >= self.min_samples_split * l[t] for t in self.l_index)
        # (2)
        m.addConstrs(a.sum('*', t) == d[t] for t in self.b_index)
        # (3)
        m.addConstrs(b[t] <= gp.quicksum(feature_max[j] * a[j, t] for j in range(self.p)) for t in self.b_index)
        m.addConstrs(b[t] >= gp.quicksum(feature_min[j] * a[j, t] for j in range(self.p)) for t in self.b_index)
        # (5)
        m.addConstrs(d[t] <= d[t//2] for t in self.b_index if t != 1)

        return m, a, b, d, l, beta, gamma

    @staticmethod
    def _calMinDist(x):
        """
        get the smallest non-zero distance of features
        """
        min_dis = []
        for j in range(x.shape[1]):
            xj = x[:,j]
            # drop duplicates
            xj = np.unique(xj)
            # sort
            xj = np.sort(xj)[::-1]
            # distance
            dis = [1]
            for i in range(len(xj)-1):
                dis.append(xj[i] - xj[i+1])
            # min distance
            min_dis.append(np.min(dis) if np.min(dis) else 1)
        return min_dis

    def _setStart(self, x, y, a, b, d, l):
        """
        set warm start from CART
        """
        # train with CART
        if self.min_samples_split > 1:
            clf = tree.DecisionTreeRegressor(max_depth=self.max_depth, min_samples_split=self.min_samples_split)
        else:
            clf = tree.DecisionTreeRegressor(max_depth=self.max_depth)
        
        clf.fit(x, y)

        # get splitting rules
        rules = self._getRules(clf)

        # fix branch node
        for t in self.b_index:
            # not split
            if rules[t].feat is None or rules[t].feat == tree._tree.TREE_UNDEFINED:
                d[t].start = 0
                b[t].start = 0
                for f in range(self.p):
                    a[f,t].start = 0
            # split
            else:
                d[t].start = 1
                b[t].start = rules[t].threshold
                for f in range(self.p):
                    if f == int(rules[t].feat):
                        a[f,t].start = 1
                    else:
                        a[f,t].start = 0

        # fix leaf nodes
        for t in self.l_index:
            # terminate early
            if rules[t].value is None:
                l[t].start = int(t % 2)
                # flows go to right
                # if t % 2:
                #     t_leaf = t
                #     while rules[t].value is None:
                #         t //= 2
                #     for k in self.labels:
                #         if k == np.argmax(rules[t].value):
                #             c[k, t_leaf].start = 1
                #         else:
                #             c[k, t_leaf].start = 0
                # nothing in left
                # else:
                #     for k in self.labels:
                #         c[k, t].start = 0
            # terminate at leaf node
            else:
                l[t].start = 1
                # for k in self.labels:
                #     if k == np.argmax(rules[t].value):
                #         c[k, t].start = 1
                #     else:
                #         c[k, t].start = 0

    def _getRules(self, clf):
        """
        get splitting rules
        """
        # node index map
        node_map = {1:0}
        for t in self.b_index:
            # terminal
            node_map[2*t] = -1
            node_map[2*t+1] = -1
            if node_map[t] == -1:
                continue
            # left
            l = clf.tree_.children_left[node_map[t]]
            node_map[2*t] = l
            # right
            r = clf.tree_.children_right[node_map[t]]
            node_map[2*t+1] = r

        # rules
        rule = namedtuple('Rules', ('feat', 'threshold', 'value'))
        rules = {}
        # branch nodes
        for t in self.b_index:
            i = node_map[t]
            if i == -1:
                r = rule(None, None, None)
            else:
                r = rule(clf.tree_.feature[i], clf.tree_.threshold[i], clf.tree_.value[i,0])
            rules[t] = r
        # leaf nodes
        for t in self.l_index:
            i = node_map[t]
            if i == -1:
                r = rule(None, None, None)
            else:
                r = rule(None, None, clf.tree_.value[i,0])
            rules[t] = r

        return rules


In [4]:
#!/usr/bin/env python3
# benchmark_iorfa_real.py

import time
import warnings
from dataclasses import dataclass
from typing import Callable, Dict, List, Optional, Tuple

import numpy as np

# --- Baselines + utilities ---
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_california_housing, load_diabetes, fetch_openml
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor


@dataclass
class DatasetSpec:
    name: str
    loader: Callable[[int], Tuple[object, np.ndarray]]  # (seed) -> (X (DataFrame-like), y)


def _onehot_dense():
    """
    OneHotEncoder that yields dense output across sklearn versions.
    """
    try:
        return OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    except TypeError:
        return OneHotEncoder(handle_unknown="ignore", sparse=False)


def make_preprocessor_for_df(X):
    """
    Builds a ColumnTransformer for numeric + categorical columns.
    Outputs:
      - sparse for baseline pipelines (fine)
      - dense for IORFA (we'll set sparse_threshold=0.0 to force dense if possible)
    """
    # Works for pandas DataFrame; if user passes numpy, treat all numeric.
    try:
        import pandas as pd  # noqa: F401
        is_df = hasattr(X, "dtypes")
    except Exception:
        is_df = hasattr(X, "dtypes")

    if not is_df:
        # All numeric
        numeric_pipe = Pipeline(
            steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
        )
        return ColumnTransformer([("num", numeric_pipe, slice(0, X.shape[1]))], remainder="drop")

    num_cols = list(X.select_dtypes(include=["number", "bool"]).columns)
    cat_cols = [c for c in X.columns if c not in num_cols]

    numeric_pipe = Pipeline(
        steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
    )
    cat_pipe = Pipeline(
        steps=[("imputer", SimpleImputer(strategy="most_frequent")), ("onehot", _onehot_dense())]
    )

    return ColumnTransformer(
        transformers=[
            ("num", numeric_pipe, num_cols),
            ("cat", cat_pipe, cat_cols),
        ],
        remainder="drop",
        sparse_threshold=0.0,  # try to keep output dense (helps IORFA)
    )


def subsample(X, y, max_n: Optional[int], seed: int):
    if max_n is None:
        return X, y
    n = len(y)
    if n <= max_n:
        return X, y
    rng = np.random.default_rng(seed)
    idx = rng.choice(n, size=max_n, replace=False)
    if hasattr(X, "iloc"):
        return X.iloc[idx], y[idx]
    return X[idx], y[idx]


def metrics(y_true, y_pred) -> Dict[str, float]:
    return {
        "RMSE": float(mean_squared_error(y_true, y_pred)),
        "MAE": float(mean_absolute_error(y_true, y_pred)),
        "R2": float(r2_score(y_true, y_pred)),
    }


def make_datasets() -> List[DatasetSpec]:
    def diabetes(seed: int):
        d = load_diabetes()
        X = d.data
        y = d.target
        rng = np.random.default_rng(seed)
        idx = rng.permutation(len(y))
        return X[idx], y[idx]

    def california(seed: int):
        d = fetch_california_housing()
        X = d.data
        y = d.target
        rng = np.random.default_rng(seed)
        idx = rng.permutation(len(y))
        return X[idx], y[idx]

    def openml_regression(data_id: int, target_col: Optional[str] = None):
        def _loader(seed: int):
            bunch = fetch_openml(data_id=data_id, as_frame=True)
            X = bunch.data
            y = bunch.target
            # If multi-target, pick one column
            if hasattr(y, "shape") and len(getattr(y, "shape", ())) == 2 and y.shape[1] > 1:
                if target_col is not None and hasattr(y, "__getitem__"):
                    yv = np.asarray(y[target_col], dtype=float).ravel()
                else:
                    yv = np.asarray(y.iloc[:, 0], dtype=float).ravel()
            else:
                yv = np.asarray(y, dtype=float).ravel()

            rng = np.random.default_rng(seed)
            idx = rng.permutation(len(yv))
            return X.iloc[idx], yv[idx]
        return _loader

    return [
        DatasetSpec("diabetes_sklearn", diabetes),
        DatasetSpec("california_housing_sklearn", california),

        # OpenML real regression datasets (by data_id)
        DatasetSpec("abalone_183", openml_regression(183)),
        DatasetSpec("wine_quality_287", openml_regression(287)),
        DatasetSpec("kin8nm_189", openml_regression(189)),
        DatasetSpec("cpu_act_197", openml_regression(197)),
        DatasetSpec("cpu_small_562", openml_regression(562)),
        DatasetSpec("yacht_hydrodynamics_42370", openml_regression(42370)),

        # Energy Efficiency has 2 targets; pick one if present (names vary; fallback is first col)
        DatasetSpec("energy_efficiency_1472", openml_regression(1472, target_col="Y1")),

        DatasetSpec("concrete_strength_44959", openml_regression(44959)),
    ]


def make_baselines(seed: int) -> Dict[str, object]:
    return {
        "LinearRegression": LinearRegression(),
        "Ridge(alpha=1.0)": Ridge(alpha=1.0, random_state=seed),
        "Lasso(alpha=0.01)": Lasso(alpha=0.01, max_iter=20000, random_state=seed),
        "CART": DecisionTreeRegressor(random_state=seed),
        "RandomForest": RandomForestRegressor(n_estimators=300, random_state=seed, n_jobs=-1),
        "ExtraTrees": ExtraTreesRegressor(n_estimators=400, random_state=seed, n_jobs=-1),
        "GradientBoosting": GradientBoostingRegressor(random_state=seed),
    }


def main():
    warnings.filterwarnings("ignore")

    # ---- knobs ----
    seed = 0
    test_size = 0.25

    # Keep datasets "not too big" in practice
    max_n_all = 5000        # subsample for ALL models (set None to use full)
    max_n_iorfa = 600       # subsample for IORFA specifically (MIQP grows fast)

    # IORFA settings (safe-ish defaults)
    iorfa_max_depth = 2
    iorfa_alpha = 0.01
    iorfa_timelimit = 60    # seconds per dataset
    iorfa_output = False

    datasets = make_datasets()
    baselines = make_baselines(seed)

    rows = []

    for ds in datasets:
        print(f"\n=== {ds.name} ===")
        try:
            X, y = ds.loader(seed)
        except Exception as e:
            print(f"SKIP (load failed): {type(e).__name__}: {e}")
            continue

        # subsample globally (keeps “not too big”)
        X, y = subsample(X, y, max_n_all, seed)

        # split
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=seed
        )

        # preprocessing
        pre = make_preprocessor_for_df(X_train)

        # ----- baselines -----
        for name, model in baselines.items():
            pipe = Pipeline([("pre", pre), ("model", model)])
            t0 = time.perf_counter()
            pipe.fit(X_train, y_train)
            fit_s = time.perf_counter() - t0
            pred = pipe.predict(X_test)
            m = metrics(y_test, pred)
            rows.append({
                "dataset": ds.name, "model": name,
                "n_train": len(y_train), "n_test": len(y_test),
                "RMSE": m["RMSE"], "MAE": m["MAE"], "R2": m["R2"],
                "fit_seconds": fit_s, "iorfa_optgap": None, "iorfa_status": None
            })
            print(f"{name:16s} RMSE={m['RMSE']:.4f}  MAE={m['MAE']:.4f}  R2={m['R2']:.4f}  fit_s={fit_s:.2f}")

        # ----- IORFA -----
        try:
            Xi_train, yi_train = subsample(X_train, y_train, max_n_iorfa, seed)
            Xi_test, yi_test = subsample(X_test, y_test, max_n_iorfa, seed + 1)

            # Fit transformer on IORFA train and transform to dense numpy
            pre_i = make_preprocessor_for_df(Xi_train)
            Xi_train_p = pre_i.fit_transform(Xi_train)
            Xi_test_p = pre_i.transform(Xi_test)

            # choose min_samples_split defensively
            mss = max(2, min(10, int(0.05 * len(yi_train))))

            # dynamic gamma bounds (tends to help solver)
            y_min, y_max = float(np.min(yi_train)), float(np.max(yi_train))
            yr = max(1.0, y_max - y_min)
            gamma_bounds = (y_min - yr, y_max + yr)

            model = optimalDecisionTreeClassifier(
                max_depth=iorfa_max_depth,
                min_samples_split=mss,
                alpha=iorfa_alpha,
                warmstart=True,
                timelimit=iorfa_timelimit,
                output=iorfa_output,
                gamma_bounds=gamma_bounds,
            )

            t0 = time.perf_counter()
            model.fit(Xi_train_p, yi_train)
            fit_s = time.perf_counter() - t0
            pred = model.predict(Xi_test_p)

            m = metrics(yi_test, pred)
            optgap = getattr(model, "optgap", None)
            status = getattr(getattr(model, "m", None), "status", None)

            rows.append({
                "dataset": ds.name,
                "model": f"IORFA(d={iorfa_max_depth},a={iorfa_alpha})",
                "n_train": len(yi_train), "n_test": len(yi_test),
                "RMSE": m["RMSE"], "MAE": m["MAE"], "R2": m["R2"],
                "fit_seconds": fit_s, "iorfa_optgap": optgap, "iorfa_status": status
            })
            print(f"{'IORFA':16s} RMSE={m['RMSE']:.4f}  MAE={m['MAE']:.4f}  R2={m['R2']:.4f}  fit_s={fit_s:.2f}  gap={optgap}  status={status}")

        except Exception as e:
            print(f"IORFA failed: {type(e).__name__}: {e}")
            rows.append({
                "dataset": ds.name,
                "model": f"IORFA(d={iorfa_max_depth},a={iorfa_alpha})",
                "n_train": None, "n_test": None,
                "RMSE": None, "MAE": None, "R2": None,
                "fit_seconds": None, "iorfa_optgap": None, "iorfa_status": None,
                "error": f"{type(e).__name__}: {e}",
            })

    # ----- save results -----
    try:
        import pandas as pd
        df = pd.DataFrame(rows)
        df.to_csv("benchmark_results_real.csv", index=False)
        print("\nSaved: benchmark_results_real.csv")
        print("\nTop rows (sorted by dataset then RMSE):")
        print(df.sort_values(["dataset", "RMSE"], na_position="last").head(30).to_string(index=False))
    except ImportError:
        print("\nInstall pandas to save CSV nicely: pip install pandas")


if __name__ == "__main__":
    main()



=== diabetes_sklearn ===
LinearRegression RMSE=3338.7253  MAE=46.4920  R2=0.4069  fit_s=0.01
Ridge(alpha=1.0) RMSE=3345.7090  MAE=46.6431  R2=0.4057  fit_s=0.01
Lasso(alpha=0.01) RMSE=3340.1610  MAE=46.5259  R2=0.4067  fit_s=0.00
CART             RMSE=7017.7748  MAE=66.6937  R2=-0.2466  fit_s=0.01
RandomForest     RMSE=3459.4941  MAE=48.1021  R2=0.3855  fit_s=0.54
ExtraTrees       RMSE=3456.6867  MAE=47.5510  R2=0.3860  fit_s=0.44
GradientBoosting RMSE=3767.9006  MAE=50.3676  R2=0.3307  fit_s=0.10
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2755804
Academic license 2755804 - for non-commercial use only - registered to pa___@mit.edu
IORFA            RMSE=3527.9729  MAE=47.9889  R2=0.3733  fit_s=60.36  gap=0.9999930547747841  status=9

=== california_housing_sklearn ===
LinearRegression RMSE=0.4791  MAE=0.4996  R2=0.6407  fit_s=0.01
Ridge(alpha=1.0) RMSE=0.4791  MAE=0.4996  R2=0.6407  fit_s=0.01
Lasso(alpha=0.01) RMSE=0.4868  MAE=0.5018  R2=0.6349 